# District Cleaning — Uttar Pradesh

This notebook standardizes **district names** for this state across:
- Enrolment data
- Demographic update data
- Biometric update data

**All data is saved back to the same cleaned files.**

In [2]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

CLEAN_DIR = Path("../../data/processed/cleaned")

enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")

for df in [enrol_df, demo_df, bio_df]:
    df["state"] = df["state"].astype(str).str.strip().str.title()
    df["district"] = df["district"].astype(str).str.strip().str.title()

print("✅ All datasets loaded and normalized (Title Case)")


✅ All datasets loaded and normalized (Title Case)


In [7]:
STATE_NAME = "Uttar Pradesh"

districts = sorted(
    set(
        enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
        .tolist()
    )
)

print(f"State: {STATE_NAME}")
print(f"Number of unique districts: {len(districts)}")

pd.DataFrame(
    {"District Name": districts}
)


State: Uttar Pradesh
Number of unique districts: 75


,District Name
0,Agra
1,Aligarh
2,Ambedkar Nagar
3,Amethi
4,Amroha
5,Auraiya
6,Ayodhya
7,Azamgarh
8,Baghpat
9,Bahraich


## District Mapping

Add mappings in **Title Case only**.

Format:
```python
DISTRICT_MAPPING = {
    "Correct District": ["Wrong Name 1", "Wrong Name 2"],
}
```

In [4]:
DISTRICT_MAPPING = {
    # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
    "Prayagraj" : ["Allahabad"],
    "Baghpat" : ["Bagpat"],
    "Bara Banki" : ["Barabanki"],
    "Ayodhya" : ["Faizabad"],
    "Amroha" : ["Jyotiba Phule Nagar"],
    "Kushinagar" : ["Kushi Nagar", "Kushinagar *"],
    "Maharajganj" : ["Mahrajganj"],
    "Rae Bareli" : ["Raebareli"],
    "Bhadohi" : ["Sant Ravidas Nagar", "Sant Ravidas Nagar Bhadohi"],
    "Shrawasti" : ["Shravasti"],
    "Siddharthnagar" : ["Siddharth Nagar"],
    "Bulandshahr" : ["Bulandshahar"],
}

def apply_mapping(df, state, mapping, label):
    total = 0
    for correct, wrongs in mapping.items():
        mask = (
            (df["state"] == state) &
            (df["district"].isin(wrongs))
        )
        count = mask.sum()
        df.loc[mask, "district"] = correct
        total += count
        if count > 0:
            print(f"✔ {label} → {correct} : {count} rows fixed")
    return total

total_fixes = 0
total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")

print(f"✅ Total fixes in {STATE_NAME}: {total_fixes}")


✔ Enrolment → Prayagraj : 2561 rows fixed
✔ Enrolment → Baghpat : 1 rows fixed
✔ Enrolment → Bara Banki : 336 rows fixed
✔ Enrolment → Ayodhya : 1972 rows fixed
✔ Enrolment → Amroha : 62 rows fixed
✔ Enrolment → Kushinagar : 7 rows fixed
✔ Enrolment → Maharajganj : 18 rows fixed
✔ Enrolment → Rae Bareli : 2 rows fixed
✔ Enrolment → Bhadohi : 887 rows fixed
✔ Enrolment → Shrawasti : 4 rows fixed
✔ Enrolment → Siddharthnagar : 6 rows fixed
✔ Enrolment → Bulandshahr : 133 rows fixed
✔ Demographic → Prayagraj : 2887 rows fixed
✔ Demographic → Baghpat : 9 rows fixed
✔ Demographic → Bara Banki : 825 rows fixed
✔ Demographic → Ayodhya : 2558 rows fixed
✔ Demographic → Amroha : 203 rows fixed
✔ Demographic → Kushinagar : 54 rows fixed
✔ Demographic → Maharajganj : 34 rows fixed
✔ Demographic → Rae Bareli : 10 rows fixed
✔ Demographic → Bhadohi : 1116 rows fixed
✔ Demographic → Bulandshahr : 263 rows fixed
✔ Biometric → Prayagraj : 3319 rows fixed
✔ Biometric → Baghpat : 6 rows fixed
✔ Biometri

In [5]:
enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)

print("💾 All cleaned files saved successfully (overwritten)")


💾 All cleaned files saved successfully (overwritten)
